
# Replacing Fundamental Modes.

As seen in the interactive notebook, as the thickness of the outer air layer of N2 increases, a lower mode comes and bumps out the fundamental.  In this notebook we visualize the modes as this occurs.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf, plotlogf_real
from ngsolve.webgui import Draw
from ngsolve import CF
from scipy.optimize import newton
from ipywidgets import interactive, FloatSlider

importing NGSolve-6.2.2203-56-g47f483033


## Propagation Constant Replacement

Below we again give the interactive plot showing the replacement behavior.

In [5]:
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, d, 1.5e-5]

H = BraggExact(ts=ts, no_mesh=True)


In [7]:
nu = 1

def det_plot(T):
    d = 4.0775e-05  # thickness of innermost region (core radius)
    ts = [d, 1e-5, T * d, 1.5e-5]
    H.ts = ts
    cutoff = H.ks[0] * H.scale
    
    Xr = np.linspace(.99985* cutoff , .999867*cutoff , num=75)
    Xi = np.linspace( -.00015, .000075, num=75)
    xr, xi = np.meshgrid(Xr, Xi)
    zs = xr + 1j * xi
    
    fig, ax1 = plt.subplots(1, 1, figsize=(12,8))
    
    fs1 = H.determinant(zs, nu=nu, outer='h2')
    ax1.contour(xr, xi, np.log(np.abs(fs1)), levels=75)
    ax1.grid(True)
    ax1.set_facecolor('grey')
    ax1.set_title('Behavior near Fundamental Mode as outer thickness changes.')
#     ax1.set_xticklabels([])
#     ax1.set_yticklabels([])
    plt.show()
    
interactive_plot = interactive(det_plot, T=FloatSlider(min=1.26,max=1.356, step=.004,
                                                       value=1.26, readout_format='.3f'))
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot

interactive(children=(FloatSlider(value=1.26, description='T', max=1.356, min=1.26, readout_format='.3f', step…

## Find the beta values for these two modes for the Ts in the slider range above.

## Visualize the modes

In [3]:
A = BraggExact(ts=ts,maxhs=[.2,.015,.07,.015])


In [4]:
Draw(CF([n for n in A.ns]), A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

In [5]:
k_low = A.k0 * A.ns[0] * A.scale
k_low

52.35987755982989

In [6]:
outer = 'h2'
nu = 1

## Below we see three modes appearing where only 1 appears normally

In [ ]:
%matplotlib inline
plotlogf(A.determinant, .9998*k_low, 1.000001*k_low, -.001,.001, nu, outer,
         iref=100, rref=100, levels=100)

## One way to discern the true fundamental is to look at the real part.

None of these are guided, but looking at the slice through the real axis shows one root is closer than the others to having zero imaginary part: this root thus has the lowest loss and is the true fundamental

In [ ]:
plotlogf_real(A.determinant, .9998*k_low, 1.00001*k_low, nu, outer)

In [ ]:
xs = np.linspace(.9998 * k_low, 1.00001 * k_low, 3000)
ys = np.abs(A.determinant(xs, nu, outer))

indx = np.argmin(ys)
guess = xs[indx]

guess

In [ ]:
guess = np.array(guess)

beta1 = newton(A.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer)))


# True Fundamental Mode

In [ ]:
Fs1 = A.all_fields(beta1, nu, outer)

## Z-Components

In [ ]:
Draw(1e0*Fs1['Ez'], A.mesh)

## Transverse field

In [ ]:
Draw(Fs1['Etv'].real, A.mesh, vectors={'grid_size':100})


# The Other fundamental modes

These are similar to the true one, but have most of their energy in the outer ring.

In [ ]:
%matplotlib inline
plotlogf(A.determinant, .9998*k_low, 1.000001*k_low, -.001,.001, nu, outer,
         iref=100, rref=100, levels=100)

### Farthest right mode:

Notably, this mode is still nearly linearly polarized (even in the ripple regions).  

(Note that this refers to the real part of the field.)

In [ ]:
guess = np.array(.99992*k_low)

beta3 = newton(A.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta3, ". Residual of determinant: ", abs(A.determinant(beta3, nu, outer)))


In [ ]:
Fs3 = A.all_fields(beta3, nu, outer)

In [ ]:
Draw(1e0*Fs3['Ez'], A.mesh)

In [ ]:
Draw(Fs3['Etv'].real, A.mesh, vectors={'grid_size':100})


### X and Y components

In [ ]:
Draw(-1e-2*Fs3['Ey'], A.mesh)


In [ ]:
Draw(-1e-2*Fs3['Ex'], A.mesh)


### Middle mode:

This is no longer close to linearly polarized, and appears to look more like a higher order mode

In [ ]:
guess = np.array(.9999*k_low)

beta2 = newton(A.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta2, ". Residual of determinant: ", abs(A.determinant(beta2, nu, outer)))


In [ ]:
Fs2 = A.all_fields(beta2, nu, outer)

In [ ]:
Draw(1e-1*Fs2['Ez'], A.mesh)

In [ ]:
Draw(1e-3*Fs2['Etv'].real, A.mesh, vectors={'grid_size':100})


### X and Y components

In [ ]:
Draw(-1e-2*Fs2['Ey'], A.mesh)


In [ ]:
Draw(-1e-2*Fs2['Ex'], A.mesh)
